In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import librosa
import librosa.display
import torch
import numpy as np
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import warnings
from tqdm.notebook import tqdm

In [ ]:
!kaggle datasets download -d chrisfilo/urbansound8k

Dataset URL: https://www.kaggle.com/datasets/chrisfilo/urbansound8k
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
urbansound8k.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip urbansound8k.zip -d urbansound8k

Archive:  urbansound8k.zip
  inflating: urbansound8k/UrbanSound8K.csv  
  inflating: urbansound8k/fold1/101415-3-0-2.wav  
  inflating: urbansound8k/fold1/101415-3-0-3.wav  
  inflating: urbansound8k/fold1/101415-3-0-8.wav  
  inflating: urbansound8k/fold1/102106-3-0-0.wav  
  inflating: urbansound8k/fold1/102305-6-0-0.wav  
  inflating: urbansound8k/fold1/102842-3-0-1.wav  
  inflating: urbansound8k/fold1/102842-3-1-0.wav  
  inflating: urbansound8k/fold1/102842-3-1-5.wav  
  inflating: urbansound8k/fold1/102842-3-1-6.wav  
  inflating: urbansound8k/fold1/103074-7-0-0.wav  
  inflating: urbansound8k/fold1/103074-7-0-1.wav  
  inflating: urbansound8k/fold1/103074-7-0-2.wav  
  inflating: urbansound8k/fold1/103074-7-1-0.wav  
  inflating: urbansound8k/fold1/103074-7-1-1.wav  
  inflating: urbansound8k/fold1/103074-7-1-2.wav  
  inflating: urbansound8k/fold1/103074-7-1-3.wav  
  inflating: urbansound8k/fold1/103074-7-1-4.wav  
  inflating: urbansound8k/fold1/103074-7-1-5.wav  
  inflatin


  inflating: urbansound8k/fold1/103074-7-4-0.wav  
  inflating: urbansound8k/fold1/103074-7-4-1.wav  
  inflating: urbansound8k/fold1/103074-7-4-2.wav  
  inflating: urbansound8k/fold1/103074-7-4-3.wav  
  inflating: urbansound8k/fold1/103074-7-4-4.wav  
  inflating: urbansound8k/fold1/103074-7-4-5.wav  
  inflating: urbansound8k/fold1/103074-7-4-6.wav  
  inflating: urbansound8k/fold1/103258-5-0-0.wav  
  inflating: urbansound8k/fold1/103258-5-0-1.wav  
  inflating: urbansound8k/fold1/103258-5-0-10.wav  
  inflating: urbansound8k/fold1/103258-5-0-11.wav  
  inflating: urbansound8k/fold1/103258-5-0-12.wav  
  inflating: urbansound8k/fold1/103258-5-0-13.wav  
  inflating: urbansound8k/fold1/103258-5-0-14.wav  
  inflating: urbansound8k/fold1/103258-5-0-15.wav  
  inflating: urbansound8k/fold1/103258-5-0-16.wav  
  inflating: urbansound8k/fold1/103258-5-0-17.wav  
  inflating: urbansound8k/fold1/103258-5-0-18.wav  
  inflating: urbansound8k/fold1/103258-5-0-19.wav  
  inflating: urbanso

In [2]:
torch.cuda.empty_cache()

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [19]:
trainPath = 'urbansound8k/'
trainData = pd.read_csv(f'{trainPath}/UrbanSound8K.csv')
trainData.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [20]:
pretext=False
if pretext:
    for i in range(len(trainData)):
        value = trainData['class'][i]
        if value == "jackhammer" or value == "engine_idling" or value == "gun_shot" or value == "drilling":
            trainData.loc[i, 'classID'] = 0
        if value == "air_conditioner" or value == 'street_music' or value == 'siren' or value == "car_horn":
            trainData.loc[i, 'classID'] = 1
        if value == 'children_playing' or value == 'dog_bark':
            trainData.loc[i, 'classID'] = 2

#trainData.to_csv('/kaggle/input/urbansound8k/UrbanSound8K.csv', index=False)
#trainData[trainData['class'] == "gun_shot"]

In [21]:
num_classes = len(set(trainData['classID'].values))
num_classes

10

In [22]:
class Dataset(Dataset):
    def __init__(self, dataframe, fold=None, val=False, test=False):
        self.fold = fold  
        all_folds = [i for i in range(1, 11)]
        test_fold = 10-fold
        all_folds.remove(test_fold)
        if test==False:
            df = dataframe[dataframe['fold'].isin(all_folds)]
            train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
            if val ==True:
                self.dataframe = val_df
            else:
                self.dataframe = train_df
               
        elif test==True:
            self.dataframe = dataframe[dataframe['fold'] == test_fold]
    def __getitem__(self, index):
        path_to_file = self.get_path_to_file(index)
        signal = self.preprocess_signal(path_to_file)

        x = np.stack([cv2.resize(signal, (224, 224)) for _ in range(3)])

        y = self.dataframe.classID.values[index]
        return torch.tensor(x, dtype=torch.float), y

    def get_path_to_file(self, index):
        return f'urbansound8k/fold{self.dataframe.fold.values[index]}/{self.dataframe.slice_file_name.values[index]}'
    def preprocess_signal(self, path_to_file):
        signal, _ = librosa.load(path_to_file, sr=16000)
        signal = librosa.feature.melspectrogram(y=signal)
        signal = librosa.power_to_db(signal)
        return signal

    def __len__(self):
        return self.dataframe.shape[0]

In [23]:
batch_size = 32
fold = 0
trainSet = Dataset(trainData, fold=fold)
valSet = Dataset(trainData, fold=fold, val=True)
testSet = Dataset(trainData, fold=fold, test=True)
trainLoader = DataLoader(trainSet, batch_size=batch_size, shuffle=True, num_workers=2)
valLoader = DataLoader(valSet , batch_size=batch_size, num_workers=2)
testLoader = DataLoader(testSet , batch_size=batch_size, num_workers=2)

print('Training set: {}, Validation set: {}, Test Set: {}'.format(len(trainSet), len(valSet), len(testSet)))

print("Folds of training set:", set(list(trainSet.dataframe["fold"].values)))
print("Folds of validation set:", set(list(valSet.dataframe["fold"].values)))
print("Folds of test set:", set(list(testSet.dataframe["fold"].values)))


Training set: 7105, Validation set: 790, Test Set: 837
Folds of training set: {1, 2, 3, 4, 5, 6, 7, 8, 9}
Folds of validation set: {1, 2, 3, 4, 5, 6, 7, 8, 9}
Folds of test set: {10}


In [24]:
print(trainSet[0][0].shape)
print(valSet[0][0].shape)
print(testSet[0][0].shape)

torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


In [25]:
class CustomResNet50(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomResNet50, self).__init__()
        # Load a pre-trained ResNet50
        self.resnet50 = models.resnet18(pretrained=True)
        # Freeze all layers in ResNet50
        for param in self.resnet50.parameters():
            param.requires_grad = True  # Set true to unfreeze

        # Get the input features of the original fully connected layer
        in_features = self.resnet50.fc.in_features

        # Replace the fully connected layer with custom layers
        self.resnet50.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.Linear(256, num_classes),
        )

        # Unfreeze the custom fully connected layers
        for param in self.resnet50.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.resnet50(x)

def get_pretext_loaded(model, pretext_path):
    backbone_weights = torch.load(pretext_path, map_location='cpu')
    model_dict = dict(model.resnet50.state_dict())
    #print(backbone_weights["model_state"].keys())
    for key, weights in backbone_weights["model_state"].items():
        a = key[9:]
        if a.find('fc') == -1: #key should not contain fc
            model_dict[a] = weights #update all keys except fc
    
    model.resnet50.load_state_dict(model_dict)
    a = model_dict['conv1.weight'][0]
    b = backbone_weights["model_state"]["resnet50.conv1.weight"][0]
    
    if np.array_equal(np.array(a), np.array(b)):
        print("Backbone Loaded")
    else:
        print("Backbone not loaded")
    return model

In [26]:
class CustomEffNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomEffNet, self).__init__()
        # Load a pre-trained EfficientNet
        self.effnet = models.efficientnet_b1(pretrained=True)
        # Freeze all layers in EfficientNet
        for param in self.effnet.parameters():
            param.requires_grad = True #Set true to unfreeze

        # Get the input features of the original classifier
        in_features = self.effnet.classifier[1].in_features

        # Replace the classifier with custom layers
        self.effnet.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
        )

        # Unfreeze the classifier layers
        for param in self.effnet.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.effnet(x)

# Create the model
#model = CustomEffNet(num_classes=50)
#model.to(device)
def get_pretext_loaded_effnet(model, prextext_path):
    backbone_weights = torch.load(prextext_path, map_location='cpu')
    model_dict = dict(model.effnet.state_dict())
    for key, weights in backbone_weights["model_state"].items():
        a = key[7:]
        if a.find('classifier') == -1:
            model_dict[a] = weights
    
    model.effnet.load_state_dict(model_dict)
    a = model_dict['features.0.0.weight'][0]
    b = backbone_weights["model_state"]["effnet.features.0.0.weight"][0]
    if np.array_equal(np.array(a), np.array(b)):
        print("Backbone Loaded")
    else:
        print("Backbone not loaded")
    return model


In [29]:
try:
    df = pd.read_csv("training_metadata_prop_resnet18.csv")
except:

    df = pd.DataFrame({"Fold": [], "Epoch": [], "Train Loss": [], "Val Loss": [], "Train Acc": [], "Val Acc": [], "Test Acc": []})

In [30]:
num_classes

10

In [31]:
from time import time 
warnings.filterwarnings("ignore")

prextext_path = f'resnet18_pretext_3_us8k_fold1.pth'
start_time = time()
folds = 10
for fold in range(3, folds):
    checkpoint_path = f'resnet18_prop_3_us8k_fold{fold+1}.pth'
    batch_size = 32
    trainSet = Dataset(trainData, fold=fold)
    valSet = Dataset(trainData, fold=fold, val=True)
    testSet = Dataset(trainData, fold=fold, test=True)
    trainLoader = DataLoader(trainSet, batch_size=batch_size, shuffle=True, num_workers = 2)
    valLoader = DataLoader(valSet , batch_size=batch_size, num_workers = 2)
    testLoader = DataLoader(testSet , batch_size=batch_size, num_workers = 2)

    print('Training set: {}, Validation set: {}, Test Set: {}'.format(len(trainSet), len(valSet), len(testSet)))
    print("Folds of training set:", set(list(trainSet.dataframe["fold"].values)))
    print("Folds of validation set:", set(list(valSet.dataframe["fold"].values)))
    print("Folds of test set:", set(list(testSet.dataframe["fold"].values)))
    print("Classfying", num_classes, "classes.")

    model = get_pretext_loaded(CustomResNet50(num_classes=num_classes), prextext_path)
    
    if torch.cuda.device_count() >= 2:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        device = torch.device("cuda")
        model = torch.nn.DataParallel(model)  # Wrap the model for multi-GPU use
        model.to(device)
    else:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)
        print("Using single GPU or CPU")
        
    epochs = 50
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    cost = torch.nn.CrossEntropyLoss()
    best_val_accuracy = 0.0
    best_val_loss = 10000
    checkpoint_path = checkpoint_path
    

    if os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        best_val_accuracy = checkpoint['best_val_accuracy']
        start_epoch = checkpoint['epoch'] + 1
        print("Resuming training from epoch: "+str(start_epoch) +" with best_val_accuracy: "+str(best_val_accuracy))
    else:
        start_epoch = 0
    
    for epoch in range(start_epoch, epochs):
        train_loss = 0
        val_loss = 0
        train_correct = 0
        val_correct = 0
        
        model.train()
        for x, y in tqdm(trainLoader):
            optimizer.zero_grad()
            x,y = x.to(device),y.to(device)
            pred = model(x)
            loss = cost(pred, y)
            train_loss += cost(pred, y).item()
            train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            for x, y in tqdm(valLoader):
                x,y = x.to(device),y.to(device)
                pred = model(x)
                loss = cost(pred, y)
                val_loss += cost(pred, y).item()
                val_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_loss = train_loss/len(trainLoader)
        val_loss = val_loss/len(valLoader)
        train_accuracy = train_correct / len(trainData)
        val_accuracy = val_correct / len(valSet)
        
        if val_loss < best_val_loss:
            print(f"Validation loss has improved from {best_val_loss} to {val_loss}")
            best_val_loss = val_loss
            
        print("epoch = %d, train_loss = %.5f, val_loss = %.5f, train_accuracy = %.5f, val_accuracy = %.5f" % (epoch, train_loss, val_loss, train_accuracy, val_accuracy))
        
        #saving the model when the val_accuracy improves
        if val_accuracy > best_val_accuracy:
            print(f"Validation Accuracy improved from {best_val_accuracy:.5f} to {val_accuracy:.5f}. Saving checkpoint.")
            best_val_accuracy = val_accuracy
            torch.save({
                'epoch': epoch,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'best_val_accuracy': best_val_accuracy
            }, checkpoint_path)
        df = df._append({"Fold": fold, "Epoch": epoch, "Train Loss": train_loss, "Val Loss": val_loss, "Train Acc": train_accuracy, "Val Acc": val_accuracy, "Test Acc": 0}, ignore_index=True)
        df.to_csv("training_metadata_prop_resnet18.csv", index=False)
        #print(df.tail(1))
    #getting final best validation accuracy here
    
    checkpoint_path=checkpoint_path
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state'])
    model.eval()
    test_correct = 0
    for x, y in tqdm(testLoader):
        x,y = x.to(device),y.to(device)
        pred = model(x)
        pred = torch.softmax(pred, axis=1)       
        test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_accuracy = test_correct / len(testSet)
    print(f"Test accuracy for fold: {fold+1} is: {test_accuracy*100}")
    #change test accuracy in dataframe where fold = fold and epoch is max
    df.loc[(df.Fold == fold) & (df.Epoch == df[df.Fold == fold].Epoch.max()), "Test Acc"] = test_accuracy
    df.to_csv("training_metadata_prop_resnet18.csv", index=False)
        
end_time = time()
total_time = end_time - start_time
print(f'Total Training Time: {total_time:.2f} seconds')

Training set: 7104, Validation set: 790, Test Set: 838
Folds of training set: {1, 2, 3, 4, 5, 6, 8, 9, 10}
Folds of validation set: {1, 2, 3, 4, 5, 6, 8, 9, 10}
Folds of test set: {7}
Classfying 10 classes.


Backbone Loaded
Using single GPU or CPU
Resuming training from epoch: 1 with best_val_accuracy: 0.9443037974683545


  0%|          | 0/222 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Validation loss has improved from 10000 to 0.1035825627297163
epoch = 1, train_loss = 0.09073, val_loss = 0.10358, train_accuracy = 0.79329, val_accuracy = 0.96456
Validation Accuracy improved from 0.94430 to 0.96456. Saving checkpoint.


...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Validation loss has improved from 10000 to 0.1286884780228138
epoch = 0, train_loss = 0.53441, val_loss = 0.12869, train_accuracy = 0.68988, val_accuracy = 0.96839
Validation Accuracy improved from 0.00000 to 0.96839. Saving checkpoint.


  0%|          | 0/223 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
checkpoint_path = f"resnet_proposed_urbansound8k_model_fold{fold+1}.pth"
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state'])
model.eval()
test_correct = 0
for x, y in tqdm(testLoader):
    x,y = x.to(device),y.to(device)
    pred = model(x)
    pred = torch.softmax(pred, axis=1)       
    test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
test_accuracy = test_correct / len(testSet)
print(f"Test accuracy for fold: {fold+1} is: {test_accuracy*100}")

  0%|          | 0/27 [00:00<?, ?it/s]

Test accuracy for fold: 4 is: 88.06682577565633
